# Pinhole Camera Model Practice Questions

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. 
The camera has a focal length `f` of 50mm. 
Calculate the position of the projection of this point on the image plane in meters.

In [3]:
import numpy as np

In [31]:
p_x = np.array([10, 5, 20]).reshape((3,1))
f=50
f/= 1000
multi = np.array([[f, 0, 0], [0, f, 0], [0, 0, 1]]) @ p_x
multi /= p_x[2]
print(multi)

[[0.025 ]
 [0.0125]
 [1.    ]]


---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. 
Assume the following intrinsic parameters: 
- `s_x` = 800 pixels/meter
- `s_y` = 800 pixels/meter
- `o_x` = 320 pixels
- `o_y` = 240 pixels
What are the pixel coordinates `(u^I, v^I)`?

In [33]:
s_x = 800
s_y = 800
o_x = 320
o_y = 240
K = np.array([[s_x, 0, o_x], [0, s_y, o_y], [0, 0, 1]])
pixels = K @ multi
print(pixels)

[[340.]
 [250.]
 [  1.]]


---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

In [45]:
K = np.array([[s_x * f, 0, o_x], [0, s_y*f, o_y], [0, 0, 1]])
print(K)

[[ 40.   0. 320.]
 [  0.  40. 240.]
 [  0.   0.   1.]]


---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t_w^c = [0, 0, 10]`. [cite_start]Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point. [cite: 14, 16, 17]

In [59]:
p_w = np.array([2, 3, 50, 1]).reshape((4,1))
rwc = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
twc = np.array([0, 0, 10]).reshape((3, 1))
extrinsic = np.hstack((rwc, twc))
temp = K @ extrinsic @ p_w
print(temp[2])
print((temp / temp[2])[:2])

[60.]
[[321.33333333]
 [242.        ]]


## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

In [55]:
u_dist = 400
v_dist = 350
a1 = 0.0001
a2 = 0.0000002
o_x = 320
o_y = 240
r2 = (u_dist - o_x) ** 2 + (v_dist - o_y) ** 2
ui = (1 + a1 * r2 + a2 * (r2 **2)) * (u_dist - o_x) + o_x
vi = (1 + a1 * r2 + a2 * (r2 **2)) * (v_dist - o_y) + o_y
print(np.array([ui, vi]))

[6024. 8083.]
